# CASE 2

## 1. Setups & Import Data

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
df=pd.read_excel("Case_2_data.xlsx", sheet_name="evals_data")
df=df[~df.student_id.isin(df[df.duplicated(["student_id", "date"])].student_id.tolist())].reset_index(drop=True)
print("Numbers of Unique student_id: ",len(df.student_id.unique()))
df.head()

Numbers of Unique student_id:  8878


,student_id,date,program,location,score_reading,score_writing,score_mathNoCalc,score_mathCalc,Total Score,District,Reading_improvement,Writing_improvement,MathNoCalc_improvement,MathCalc_improvement,total_improvement
0,128368,2016-05-30,intake,NaN,741,64,304,431,1540,Camas,NaN,NaN,NaN,NaN,NaN
1,128368,2016-06-24,skills,center,764,66,320,446,1596,Camas,0.031039,0.031250,0.052632,0.034803,0.036364
2,128374,2016-08-30,intake,NaN,561,48,140,528,1277,Wilsonville,NaN,NaN,NaN,NaN,NaN
3,128374,2016-09-22,skills,center,570,50,148,542,1310,Wilsonville,0.016043,0.041667,0.057143,0.026515,0.025842
4,128375,2016-07-20,intake,NaN,627,64,525,486,1702,Ridgefield,NaN,NaN,NaN,NaN,NaN


## 2. Overall Performance

### 2.1 Avg Intake Scores

In [3]:
avg_intake=df[["score_reading","score_writing","score_mathNoCalc","score_mathCalc","Total Score"]].mean()
avg_intake

score_reading        605.109523
score_writing         51.025754
score_mathNoCalc     493.888095
score_mathCalc       560.755994
Total Score         1710.779366
dtype: float64

### 2.2 Improvement Ranges

In [4]:
min_max_improv_prog=pd.concat([df[['program','Reading_improvement', 'Writing_improvement',
       'MathNoCalc_improvement', 'MathCalc_improvement', 'total_improvement']].groupby("program").min(),
df[['program','Reading_improvement', 'Writing_improvement',
       'MathNoCalc_improvement', 'MathCalc_improvement', 'total_improvement']].groupby("program").max()])
#min_max_improv_prog.to_csv("min_max_improv_prog.csv")
min_max_improv_prog

,Reading_improvement,Writing_improvement,MathNoCalc_improvement,MathCalc_improvement,total_improvement
program,,,,,
intake,NaN,NaN,NaN,NaN,NaN
refresh,-0.034026,-0.055556,-0.025114,-0.033033,-0.015817
skills,-0.024161,-0.125000,-0.024283,-0.026432,-0.011460
tutoring,-0.010292,0.000000,0.003571,-0.001880,0.009944
intake,NaN,NaN,NaN,NaN,NaN
refresh,0.062583,0.100000,0.072093,0.075117,0.067600
skills,0.076807,0.333333,0.094637,0.250000,0.076246
tutoring,0.107981,0.500000,0.147239,0.114094,0.097185


### 2.3 Improvements by Districts

In [5]:
dstrct_imprv=df.iloc[:,-6:].dropna().groupby('District').mean()
dstrct_imprv.to_csv("dstrct_imprv.csv")
dstrct_imprv

,Reading_improvement,Writing_improvement,MathNoCalc_improvement,MathCalc_improvement,total_improvement
District,,,,,
Beaverton,0.021910,0.026841,0.036999,0.027338,0.027937
Camas,0.022373,0.026838,0.037782,0.027381,0.028415
Lake Oswego,0.026147,0.031222,0.042058,0.031375,0.032456
Ridgefield,0.020893,0.025853,0.036045,0.026487,0.027029
Riverdale,0.020036,0.024666,0.034915,0.025471,0.026023
Sherwood,0.022347,0.026510,0.037539,0.027880,0.028414
St. Paul,0.022616,0.026838,0.037250,0.027268,0.028248
Wilsonville,0.032191,0.036506,0.047477,0.036939,0.038030


### 2.4 Improvements by Programs

In [6]:
prgrm_imprv=df[["program","Reading_improvement","Writing_improvement","MathNoCalc_improvement","MathCalc_improvement","total_improvement"]]\
    .dropna().groupby("program").mean()
prgrm_imprv

,Reading_improvement,Writing_improvement,MathNoCalc_improvement,MathCalc_improvement,total_improvement
program,,,,,
refresh,0.009954,0.010930,0.017334,0.011847,0.012708
skills,0.023639,0.028793,0.039704,0.029600,0.030003
tutoring,0.037883,0.044243,0.057994,0.044165,0.045873


###  2.5 Total Improvements Since the Beginning

In [7]:

start_end=pd.concat([df.loc[df.program=="intake",:"District"], df.groupby("student_id").apply(lambda x: x.loc[(x.date==max(x.date)),:"District"])])\
    .reset_index(drop=True)

total_improvement=start_end.groupby("student_id").apply(lambda x:\
    x.loc[x.date==max(x.date),"score_reading":"Total Score"].squeeze()/x.loc[x.date==min(x.date),"score_reading":"Total Score"].squeeze()-1)\
        .reset_index()
total_improvement=total_improvement.rename(columns={"score_reading": "reading", "score_writing": "writing","score_mathNoCalc":"mathNoCalc",\
    "score_mathCalc":"mathCalc","Total Score":"total"})
           
adding_loc_data=start_end.dropna(subset=["location"])[["student_id","location","District","date","program"]]
total_improvement=total_improvement.merge(adding_loc_data, how="left",on="student_id")
total_improvement.head()

,student_id,reading,writing,mathNoCalc,mathCalc,total,location,District,date,program
0,128368,0.031039,0.031250,0.052632,0.034803,0.036364,center,Camas,2016-06-24,skills
1,128374,0.016043,0.041667,0.057143,0.026515,0.025842,center,Wilsonville,2016-09-22,skills
2,128375,0.030303,0.046875,0.112381,0.088477,0.072855,center,Ridgefield,2016-10-31,skills
3,128387,0.076446,0.068966,0.081081,0.099778,0.085582,center,Lake Oswego,2016-02-09,tutoring
4,128390,0.078464,0.040816,0.153716,0.070845,0.097264,online,St. Paul,2018-09-01,tutoring


## 3. Regression Analysis

### 3.1 Improvements by Each Program

#### Formatting Data

In [8]:
prg = pd.DataFrame(columns=["skills","tutoring","refresh","reading","writing","mathNoCalc","mathCalc","total"])
df=df.sort_values(by=['student_id', 'date'], ascending=True).reset_index(drop=True)
for i in range (1,len(df)): 
    if df.loc[i,"program"]  != "intake":
        prg.loc[i-1,df.loc[i,"program"]]=1
        prg.loc[i-1,"reading"]=df.loc[i,"score_reading"]/df.loc[i-1,"score_reading"]-1
        prg.loc[i-1,"writing"]=df.loc[i,"score_writing"]/df.loc[i-1,"score_writing"]-1
        prg.loc[i-1,"mathNoCalc"]=df.loc[i,"score_mathNoCalc"]/df.loc[i-1,"score_mathNoCalc"]-1
        prg.loc[i-1,"mathCalc"]=df.loc[i,"score_mathCalc"]/df.loc[i-1,"score_mathCalc"]-1
        prg.loc[i-1,"total"]=df.loc[i,"Total Score"]/df.loc[i-1,"Total Score"]-1
prg=prg.fillna(0)
prg.head()

,skills,tutoring,refresh,reading,writing,mathNoCalc,mathCalc,total
0,1,0,0,0.031039,0.031250,0.052632,0.034803,0.036364
2,1,0,0,0.016043,0.041667,0.057143,0.026515,0.025842
4,1,0,0,0.015949,0.000000,0.041905,0.030864,0.027615
5,1,0,0,0.028257,0.062500,0.065814,0.049900,0.047456
6,1,0,0,-0.013740,-0.014706,0.001715,0.005703,-0.003275


#### Improvement~Program Regression

In [9]:
for i in ["skills","tutoring","refresh"]:
    for j in ["reading","writing","mathNoCalc","mathCalc","total"]:
        reg=smf.ols("{}~{}".format(j,i), prg).fit()
        print("SUMMARY of {}_improvement ~ {}".format(j,i))
        print(reg.summary())
        print("\n")
        print("+"*100)
        print("\n")

SUMMARY of reading_improvement ~ skills
                            OLS Regression Results                            
Dep. Variable:                reading   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     24.39
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           7.96e-07
Time:                        19:06:21   Log-Likelihood:                 45521.
No. Observations:               17564   AIC:                        -9.104e+04
Df Residuals:                   17562   BIC:                        -9.102e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   

### 3.2 Improvement by Online/Center

online=0; center=1

#### Formatting Data

In [10]:
def check_mix_loc(df):
    df=df[(df.program!="skills")&(df.program!="intake")  & (pd.DatetimeIndex(df['date']).year>2017)]
    if (("center" in df.location.tolist()) & ("center" in df.location.tolist())):
        return df
mix_loc=df.groupby("student_id").apply(lambda x: check_mix_loc(x)).reset_index(drop=True)
mix_loc.loc[mix_loc.location=="center","location"]=1
mix_loc.loc[mix_loc.location=="online","location"]=0
mix_loc.head()

,student_id,date,program,location,score_reading,score_writing,score_mathNoCalc,score_mathCalc,Total Score,District,Reading_improvement,Writing_improvement,MathNoCalc_improvement,MathCalc_improvement,total_improvement
0,128398,2018-12-16,refresh,1,530,66,348,548,1492,St. Paul,0.001890,0.000000,0.011628,0.020484,0.010840
1,128437,2018-04-27,tutoring,1,696,69,390,524,1679,Wilsonville,0.070769,0.078125,0.104816,0.087137,0.083925
2,128605,2018-09-18,tutoring,1,733,63,677,638,2111,St. Paul,0.071637,0.086207,0.090177,0.065109,0.075943
3,128605,2018-12-08,refresh,0,771,65,719,670,2225,St. Paul,0.014474,0.015625,0.030086,0.022901,0.022049
4,128624,2018-11-27,tutoring,1,724,59,570,731,2084,Beaverton,0.072593,0.072727,0.077505,0.067153,0.072016


#### Overall_Improvement~Online/Center

In [11]:
for i in ['Reading_improvement', 'Writing_improvement',
       'MathNoCalc_improvement', 'MathCalc_improvement', 'total_improvement']:
    print("SUMMARY of {} ~ location".format(i))
    reg_refresh=smf.ols("{}~location".format(i),mix_loc).fit()
    print(reg_refresh.summary())
    print("\n")
    print("+"*100)
    print("\n")      

SUMMARY of Reading_improvement ~ location
                             OLS Regression Results                            
Dep. Variable:     Reading_improvement   R-squared:                       0.090
Model:                             OLS   Adj. R-squared:                  0.090
Method:                  Least Squares   F-statistic:                     154.3
Date:                 Thu, 11 Nov 2021   Prob (F-statistic):           7.78e-34
Time:                         19:06:29   Log-Likelihood:                 3300.1
No. Observations:                 1553   AIC:                            -6596.
Df Residuals:                     1551   BIC:                            -6586.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

#### Seperate Overall_Improvement~Online/Center by Programs(Refresh/Tutoring)

In [12]:
df_refresh=mix_loc[mix_loc.program=="refresh"]
df_tutoring=mix_loc[mix_loc.program=="tutoring"]
for i in ['Reading_improvement', 'Writing_improvement',
       'MathNoCalc_improvement', 'MathCalc_improvement', 'total_improvement']:
    
    print("SUMMARY of {} ~ location in REFRESH".format(i))
    reg_refresh=smf.ols("{}~location".format(i), df_refresh).fit()
    print(reg_refresh.summary())
    print("\n")
    print("+"*100)
    print("\n")

    print("SUMMARY of {} ~ location in TUTORING".format(i))
    reg_tutoring=smf.ols("{}~location".format(i), df_tutoring).fit()
    print(reg_tutoring.summary())
    print("\n")
    print("+"*100)
    print("\n")

SUMMARY of Reading_improvement ~ location in REFRESH
                             OLS Regression Results                            
Dep. Variable:     Reading_improvement   R-squared:                       0.036
Model:                             OLS   Adj. R-squared:                  0.034
Method:                  Least Squares   F-statistic:                     24.28
Date:                 Thu, 11 Nov 2021   Prob (F-statistic):           1.05e-06
Time:                         19:06:29   Log-Likelihood:                 1961.8
No. Observations:                  657   AIC:                            -3920.
Df Residuals:                      655   BIC:                            -3911.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

### 3.3 Improvement by Programs Taken

In [13]:
prg_comp = pd.DataFrame(columns=["student_id","skills","tutoring","refresh"])
for i in df.student_id:
    prg_rcd=df.loc[df.student_id==i,"program"].tolist()
    prg_comp.loc[i,"student_id"]=i
    prg_comp.loc[i,"skills"]=int("skills" in prg_rcd)
    prg_comp.loc[i,"tutoring"]=int("tutoring" in prg_rcd)
    prg_comp.loc[i,"refresh"]=int("refresh" in prg_rcd)
prg_comp=prg_comp.merge(total_improvement.iloc[:,:-4], how="left", on ="student_id")
#prg_comp.to_csv("prg_comp.csv")
prg_comp

,student_id,skills,tutoring,refresh,reading,writing,mathNoCalc,mathCalc,total
0,128368,1,0,0,0.031039,0.031250,0.052632,0.034803,0.036364
1,128374,1,0,0,0.016043,0.041667,0.057143,0.026515,0.025842
2,128375,1,0,0,0.030303,0.046875,0.112381,0.088477,0.072855
3,128387,0,1,0,0.076446,0.068966,0.081081,0.099778,0.085582
4,128390,1,1,0,0.078464,0.040816,0.153716,0.070845,0.097264
...,...,...,...,...,...,...,...,...,...
8873,179191,1,1,0,0.067083,0.115385,0.103118,0.084677,0.083437
8874,179210,1,0,1,-0.001618,0.052632,0.049724,0.069277,0.039851
8875,179235,1,1,0,0.112853,0.170732,0.165165,0.163793,0.140441
8876,179239,0,1,0,0.066667,0.106061,0.115132,0.116236,0.094784
